# Mixture Model

## Description

$F_{D}^{S, T}$ is a multinomial distribution over {1, 2, 3, 4, 5}.

$F_{L}^{S, T}$ is a multinomial distribution over {1, 2, 3, 4, 5}.

where $D$ is the dictator game, $L$ is the lying-dictator game, $S$ is a participant's location, Chapman (C) or Wuhan (W), and $T$ is the recipient's location, Chapman (C) or Wuhan (W). 

$F_{U}$ is the uniform distribution over {1, 2, 3, 4, 5}.

$L(X) = L(F_{D}^{S, T}(X_{D})) + \alpha L(F_{L}^{S, T}(X_{L})) + (1-\alpha)L(F_{U}(X_{L}))$

## Imports

In [75]:
import numpy as np
import pandas as pd
from scipy.stats import multinomial
from scipy.optimize import minimize

In [76]:
np.set_printoptions(suppress=True)

## Load Data

In [77]:
altruism_df = pd.read_csv('/Users/aaronberman/Desktop/Altruism & Lying Aversion/data/altruism_all_data_df.csv', index_col=0)
columns_to_drop = ['StartDate', 'EndDate', 'Status', 'Progress', 'Finished', 'RecordedDate', 'DistributionChannel', 'UserLanguage',
                   'Consent', 'Min Tokens', 'Max Tokens', 'SM', 'University', 'understand', 'session_id', 'recruiter_id', 'public_id',
                   'email', 'Total']
altruism_df = altruism_df.drop(columns_to_drop, axis=1)

In [78]:
dictator_df = altruism_df[altruism_df['Treatment'] == 'Dict']
dictator_outcomes = dictator_df['decision']

In [79]:
lie_df = altruism_df[altruism_df['Treatment'] == 'Lie']
lying_outcomes = lie_df['decision']

## Dictator and Lying-Dictator Mixture 

### Multi-Nomial Component Distributions

In [80]:
def multinomial_prob(probs, outcome):
    return probs[outcome-1]  # outcome is 1-based index, Python uses 0-based

def uniform_prob():
    return 1/5


### Likelihood Function

log-likelihood by summing the probabilities of observed outcomes.

In [81]:
def likelihood(params, outcomes_dictator, outcomes_lying):
    alpha = params[0]
    probs_d = params[1:6]  # Probabilities for F_{D}
    probs_l = params[6:11] # Probabilities for F_{L}

    ll_dictator = np.sum(np.log([multinomial_prob(probs_d, x) for x in outcomes_dictator]))
    ll_lying = np.sum(np.log([alpha * multinomial_prob(probs_l, x) + (1 - alpha) * uniform_prob() for x in outcomes_lying]))

    return -(ll_dictator + ll_lying)  # Negative since using the minimize function from scipy.stats


### Optimization

In [82]:
initial_guess = [0.5] + [1/5]*5 + [1/5]*5  # Initialize alpha and probabilities
bounds = [(0, 1)] + [(0, 1)]*10
constraints = ({'type': 'eq', 'fun': lambda x: sum(x[1:6]) - 1},   # Ensure probabilities sum to 1
               {'type': 'eq', 'fun': lambda x: sum(x[6:11]) - 1})

result = minimize(likelihood, initial_guess, args=(dictator_outcomes, lying_outcomes), bounds=bounds, constraints=constraints)

/var/folders/ry/bf84d3cd54jctn_cwy_5666m0000gn/T/ipykernel_96996/1715454999.py:6: RuntimeWarning: divide by zero encountered in log
  ll_dictator = np.sum(np.log([multinomial_prob(probs_d, x) for x in outcomes_dictator]))
/var/folders/ry/bf84d3cd54jctn_cwy_5666m0000gn/T/ipykernel_96996/1715454999.py:7: RuntimeWarning: divide by zero encountered in log
  ll_lying = np.sum(np.log([alpha * multinomial_prob(probs_l, x) + (1 - alpha) * uniform_prob() for x in outcomes_lying]))


In [83]:
print('Optimized alpha:', result.x[0])
print('Optimized probs for D:', result.x[1:6])
print('Optimized probs for L:', result.x[6:11])

Optimized alpha: 0.7832092716781021
Optimized probs for D: [0.3509643  0.2403852  0.37019333 0.02403272 0.01442446]
Optimized probs for L: [0.28299848 0.2702293  0.28299514 0.09785206 0.06592502]


### Preventing Zero-Division Error

In [84]:
# Define the modified likelihood function with epsilon to prevent log(0) errors
def likelihood_with_epsilon(params, outcomes_dictator, outcomes_lying, eps=1e-8):
    alpha = params[0]
    probs_d = params[1:6]  # Probabilities for F_{D}
    probs_l = params[6:11] # Probabilities for F_{L}

    # Add epsilon to the probabilities to prevent log(0)
    ll_dictator = np.sum(np.log([multinomial_prob(probs_d, x) + eps for x in outcomes_dictator]))
    ll_lying = np.sum(np.log([alpha * (multinomial_prob(probs_l, x) + eps) + (1 - alpha) * (uniform_prob() + eps) for x in outcomes_lying]))

    return -(ll_dictator + ll_lying)  # Negative since using the minimize function from scipy.stats

In [98]:
# Set the epsilon value
epsilon = 1e-8

# Modify the bounds to include epsilon to ensure probabilities cannot be exactly 0
bounds_with_epsilon = [(0, 1)] + [(epsilon, 1)]*10

# Redefine the constraints to ensure that the probabilities sum up to 1
constraints_with_epsilon = [
    {'type': 'eq', 'fun': lambda x: sum(x[1:6]) - 1},
    {'type': 'eq', 'fun': lambda x: sum(x[6:11]) - 1}
]

# Use the initial guess that ensures all probabilities are above epsilon
initial_guess_with_epsilon = [0.5] + [1/5]*5 + [1/5]*5

# Run the optimization with the modified likelihood function and new bounds
result_with_epsilon = minimize(likelihood_with_epsilon, initial_guess_with_epsilon, 
                               args=(dictator_outcomes, lying_outcomes, epsilon), 
                               bounds=bounds_with_epsilon, 
                               constraints=constraints_with_epsilon)


optimized_alpha = result_with_epsilon.x[0]
optimized_probs_d = result_with_epsilon.x[1:6]
optimized_probs_l = result_with_epsilon.x[6:11]

result_with_epsilon.success

True

In [99]:
print('Optimized Alpha:', optimized_alpha)
print('Optimized probabilities for the Dictator Game:', optimized_probs_d)
print('Optimized probabilities for the Lying-Dictator Game:', optimized_probs_l)

Optimized Alpha: 0.7359111945445413
Optimized probabilities for the Dictator Game: [0.3509658  0.2403865  0.37019148 0.02403229 0.01442394]
Optimized probabilities for the Lying-Dictator Game: [0.28832671 0.27473969 0.28832708 0.09128992 0.0573166 ]


## Dictator and Lying-Dictator Mixture including Dictator and Receiver Locations (University affiliations)

### Prepare data for the more complex model

In [87]:
# Recoding the 'Treatment', 'Location', and 'Partner' columns
altruism_df['Treatment'] = altruism_df['Treatment'].map({'Dict': 'D', 'Lie': 'L'})
altruism_df['Location'] = altruism_df['Location'].map({'Chapman': 'C', 'Wuhan': 'W'})
altruism_df['Partner'] = altruism_df['Partner'].map({'Chapman': 'C', 'Wuhan': 'W'})

# Extracting the relevant columns for the mixture model
data_for_model = altruism_df[['decision', 'Treatment', 'Location', 'Partner']]

# Preview the modified data_for_model DataFrame
data_for_model.head()

,decision,Treatment,Location,Partner
0,1,D,C,C
1,1,D,C,C
2,3,D,C,C
3,1,L,C,C
4,1,L,C,C


### Distributions

same as previous

### Likelihood Function

In [88]:
# Create a dictionary mapping (S, T) to probability slices as defined by ''Treatment', 'Partner', and 'Location'
prob_indices = {
    ('D', 'C', 'C'): slice(1, 6),
    ('D', 'C', 'W'): slice(6, 11),
    ('D', 'W', 'C'): slice(11, 16),
    ('D', 'W', 'W'): slice(16, 21),
    ('L', 'C', 'C'): slice(21, 26),
    ('L', 'C', 'W'): slice(26, 31),
    ('L', 'W', 'C'): slice(31, 36),
    ('L', 'W', 'W'): slice(36, 41),
}

In [89]:
def likelihood_complex(params, data):
    alpha = params[0]
    ll = 0
    for index, row in data.iterrows():
        probs = params[prob_indices[(row['Treatment'], row['Location'], row['Partner'])]]
        if row['Treatment'] == 'D':
            ll += np.log(multinomial_prob(probs, row['decision']))
        elif row['Treatment'] == 'L':
            ll += np.log(alpha * multinomial_prob(probs, row['decision']) + (1 - alpha) * uniform_prob())
    return -ll  # Minimize the negative log-likelihood

### Optimization

In [90]:
constraints = [
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('D', 'C', 'C')]]) - 1},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('D', 'C', 'W')]]) - 1},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('D', 'W', 'C')]]) - 1},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('D', 'W', 'W')]]) - 1},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('L', 'C', 'C')]]) - 1},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('L', 'C', 'W')]]) - 1},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('L', 'W', 'C')]]) - 1},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('L', 'W', 'W')]]) - 1},
]

In [91]:
# Set up the optimization
initial_guess = [0.5] + [1/5]*40  # alpha plus eight sets of probabilities for all 'Treatment', 'Location', and 'Partner' combinations
bounds = [(0, 1)] + [(0, 1)]*40

# Run the optimization
result = minimize(likelihood_complex, initial_guess, args=(data_for_model,), bounds=bounds, constraints=constraints, options={'disp': True})

# Optimization output
result.x, result.success

/var/folders/ry/bf84d3cd54jctn_cwy_5666m0000gn/T/ipykernel_96996/2451862824.py:7: RuntimeWarning: divide by zero encountered in log
  ll += np.log(multinomial_prob(probs, row['decision']))
/var/folders/ry/bf84d3cd54jctn_cwy_5666m0000gn/T/ipykernel_96996/2451862824.py:9: RuntimeWarning: divide by zero encountered in log
  ll += np.log(alpha * multinomial_prob(probs, row['decision']) + (1 - alpha) * uniform_prob())


Optimization terminated successfully    (Exit mode 0)
            Current function value: 543.8944012119003
            Iterations: 51
            Function evaluations: 2194
            Gradient evaluations: 51


(array([0.88267003, 0.37038181, 0.1851847 , 0.40739729, 0.03703619,
        0.        , 0.3333412 , 0.2222237 , 0.37035952, 0.03703725,
        0.03703833, 0.33333817, 0.2941061 , 0.33334135, 0.01960764,
        0.01960674, 0.3673515 , 0.26529949, 0.36734901, 0.        ,
        0.        , 0.22774729, 0.34334688, 0.11214202, 0.18150411,
        0.1352597 , 0.18150089, 0.25086599, 0.34334836, 0.1352653 ,
        0.08901947, 0.35105909, 0.13863112, 0.46907146, 0.02061927,
        0.02061906, 0.33007582, 0.30910057, 0.1832128 , 0.09929432,
        0.07831649]),
 True)

In [92]:
# Results
optimized_parameters = np.array(
    [0.88267003, 0.37038181, 0.1851847 , 0.40739729, 0.03703619,
        0.        , 0.3333412 , 0.2222237 , 0.37035952, 0.03703725,
        0.03703833, 0.33333817, 0.2941061 , 0.33334135, 0.01960764,
        0.01960674, 0.3673515 , 0.26529949, 0.36734901, 0.        ,
        0.        , 0.22774729, 0.34334688, 0.11214202, 0.18150411,
        0.1352597 , 0.18150089, 0.25086599, 0.34334836, 0.1352653 ,
        0.08901947, 0.35105909, 0.13863112, 0.46907146, 0.02061927,
        0.02061906, 0.33007582, 0.30910057, 0.1832128 , 0.09929432,
        0.07831649
        ]
        )

# Alpha
optimized_alpha = optimized_parameters[0]

# Probability mapping based on probability slices
prob_mapping = {
    'D_CC': optimized_parameters[1:6],
    'D_CW': optimized_parameters[6:11],
    'D_WC': optimized_parameters[11:16],
    'D_WW': optimized_parameters[16:21],
    'L_CC': optimized_parameters[21:26],
    'L_CW': optimized_parameters[26:31],
    'L_WC': optimized_parameters[31:36],
    'L_WW': optimized_parameters[36:41],
}

# Formatted Print
output = f"Optimized alpha: {optimized_alpha:.4f}\n"
for treatment_location, probs in prob_mapping.items():
    output += f"Optimized probabilities for {treatment_location}: {probs.round(4)}\n"

print(output)

Optimized alpha: 0.8827
Optimized probabilities for D_CC: [0.3704 0.1852 0.4074 0.037  0.    ]
Optimized probabilities for D_CW: [0.3333 0.2222 0.3704 0.037  0.037 ]
Optimized probabilities for D_WC: [0.3333 0.2941 0.3333 0.0196 0.0196]
Optimized probabilities for D_WW: [0.3674 0.2653 0.3673 0.     0.    ]
Optimized probabilities for L_CC: [0.2277 0.3433 0.1121 0.1815 0.1353]
Optimized probabilities for L_CW: [0.1815 0.2509 0.3433 0.1353 0.089 ]
Optimized probabilities for L_WC: [0.3511 0.1386 0.4691 0.0206 0.0206]
Optimized probabilities for L_WW: [0.3301 0.3091 0.1832 0.0993 0.0783]



### Removing divide by zero possibility

In [93]:
# Adjust the bounds
eps = 1e-8
bounds = [(0, 1)] + [(eps, 1)]*40  # Ensure probabilities cannot be exactly 0, prevent log(0)

prob_sum = 1 - 5 * eps  # Adjust for the minimum value epsilon

constraints_with_epsilon = [
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('D', 'C', 'C')]]) - prob_sum},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('D', 'C', 'W')]]) - prob_sum},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('D', 'W', 'C')]]) - prob_sum},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('D', 'W', 'W')]]) - prob_sum},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('L', 'C', 'C')]]) - prob_sum},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('L', 'C', 'W')]]) - prob_sum},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('L', 'W', 'C')]]) - prob_sum},
    {'type': 'eq', 'fun': lambda x: sum(x[prob_indices[('L', 'W', 'W')]]) - prob_sum},
]



In [94]:
def likelihood_debug(params, data):
    alpha = params[0]
    ll = 0
    for index, row in data.iterrows():
        probs = params[prob_indices[(row['Treatment'], row['Location'], row['Partner'])]]
        outcome_prob = multinomial_prob(probs, row['decision'])
        if row['Treatment'] == 'D':
            # Add an epsilon to the outcome probability to avoid log(0)
            ll += np.log(outcome_prob + eps)
        elif row['Treatment'] == 'L':
            # Add an epsilon to the mixture probability to avoid log(0)
            ll += np.log(alpha * outcome_prob + (1 - alpha) * uniform_prob() + eps)
    return -ll  # Minimize the negative log-likelihood

In [95]:
# Set up the optimization
initial_guess = [0.5] + [1/5]*40  # alpha plus eight sets of probabilities for all 'Treatment', 'Location', and 'Partner' combinations

# Run the optimization
result = minimize(likelihood_debug, initial_guess, args=(data_for_model,), bounds=bounds, constraints=constraints_with_epsilon, options={'disp': True})

# Optimization output
result.x, result.success

/Users/aaronberman/opt/anaconda3/envs/MGPGG_DA/lib/python3.9/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: 543.8944037661754
            Iterations: 49
            Function evaluations: 2112
            Gradient evaluations: 49


(array([0.91598148, 0.37038659, 0.18518655, 0.40738301, 0.0370438 ,
        0.00000001, 0.33334934, 0.22221989, 0.37035954, 0.03703171,
        0.03703946, 0.33335199, 0.29409428, 0.33334098, 0.01960674,
        0.01960596, 0.36735411, 0.26529096, 0.36735486, 0.00000001,
        0.00000001, 0.22672367, 0.33813303, 0.1153474 , 0.18217929,
        0.13761656, 0.18217374, 0.2490276 , 0.33813477, 0.13761281,
        0.09305104, 0.34555923, 0.1408579 , 0.45930226, 0.02714103,
        0.02713953, 0.32534037, 0.30512554, 0.18383243, 0.10297145,
        0.08273016]),
 True)

In [96]:
# Results
optimized_parameters_epsilon = np.array([
    0.91598148, 0.37038659, 0.18518655, 0.40738301, 0.0370438,
    0.00000001, 0.33334934, 0.22221989, 0.37035954, 0.03703171,
    0.03703946, 0.33335199, 0.29409428, 0.33334098, 0.01960674,
    0.01960596, 0.36735411, 0.26529096, 0.36735486, 0.00000001,
    0.00000001, 0.22672367, 0.33813303, 0.1153474 , 0.18217929,
    0.13761656, 0.18217374, 0.2490276 , 0.33813477, 0.13761281,
    0.09305104, 0.34555923, 0.1408579 , 0.45930226, 0.02714103,
    0.02713953, 0.32534037, 0.30512554, 0.18383243, 0.10297145,
    0.08273016
])

# Alpha
optimized_alpha_epsilon = optimized_parameters_epsilon[0]

# Probability mapping based on probability slices
prob_mapping_epsilon = {
    'D_CC': optimized_parameters_epsilon[1:6],
    'D_CW': optimized_parameters_epsilon[6:11],
    'D_WC': optimized_parameters_epsilon[11:16],
    'D_WW': optimized_parameters_epsilon[16:21],
    'L_CC': optimized_parameters_epsilon[21:26],
    'L_CW': optimized_parameters_epsilon[26:31],
    'L_WC': optimized_parameters_epsilon[31:36],
    'L_WW': optimized_parameters_epsilon[36:41],
}

# Formatted print
output = f"Optimized alpha: {optimized_alpha_epsilon:.4f}\n"
for treatment_location, probs in prob_mapping_epsilon.items():
    output += f"Optimized probabilities for {treatment_location}: {probs.round(4)}\n"

print(output)


Optimized alpha: 0.9160
Optimized probabilities for D_CC: [0.3704 0.1852 0.4074 0.037  0.    ]
Optimized probabilities for D_CW: [0.3333 0.2222 0.3704 0.037  0.037 ]
Optimized probabilities for D_WC: [0.3334 0.2941 0.3333 0.0196 0.0196]
Optimized probabilities for D_WW: [0.3674 0.2653 0.3674 0.     0.    ]
Optimized probabilities for L_CC: [0.2267 0.3381 0.1153 0.1822 0.1376]
Optimized probabilities for L_CW: [0.1822 0.249  0.3381 0.1376 0.0931]
Optimized probabilities for L_WC: [0.3456 0.1409 0.4593 0.0271 0.0271]
Optimized probabilities for L_WW: [0.3253 0.3051 0.1838 0.103  0.0827]

